In [ ]:
# !pip install torch torchvision torchaudio
# pip install "numpy<2"
# !pip install transformers
# import sys
# !{sys.executable} -m pip install transformers
# !{sys.executable} -m pip install "numpy<2"
# !{sys.executable} -m pip install torch torchvision torchaudio
# !{sys.executable} -m pip install pandas

In [14]:
import torch
import sys
import pandas as pd
from transformers import pipeline
import re

In [25]:
# Read the CSV file into a DataFrame
df = pd.read_csv('../data/Hospital_data_cleaned.csv') 

In [26]:
item_agg = (
    df[df['SALES_DT'] < '2024-04-01']
    .groupby('ITEM_NM')['SALES_DT']
    .nunique()
    .reset_index(name='distinct_sales_dates')
    .query('distinct_sales_dates > 10')
)
valid_items = item_agg['ITEM_NM']
df_filtered = df[df['ITEM_NM'].isin(valid_items)]


def clean_text(text):
    text = text.lower()
    text = re.sub(r'[^a-z\s]', '', text)
    return text

cleaned = df_filtered['ITEM_NM'].apply(clean_text)
tokenized = cleaned.apply(lambda x: x.split())

0                                     [tender]
1                                     [tender]
2                                     [tender]
3                                     [tender]
4                                     [tender]
                          ...                 
704918                     [water, dasani, oz]
704919                     [water, dasani, oz]
704920         [water, smartwater, oz, bottle]
704921    [water, vitaminwater, zero, xxx, oz]
704922          [wicked, protein, bar, refrig]
Name: ITEM_NM, Length: 683509, dtype: object

In [28]:
# Predefined categories
labels = ["Beverage", "Dessert", "Sandwich", "Breakfast", "Salad", "Fruit", "Vegetable", "Side"]

# Load classifier
classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")

# Assuming `tokenized` is a pandas Series of lists
# Loop through each row's tokens and classify each word
classification_results = []

for tokens in tokenized:
    for token in tokens:
        result = classifier(token, labels)
        classification_results.append({
            "token": token,
            "label": result['labels'][0],         # top prediction
            "score": result['scores'][0]          # top confidence
        })

results_df = pd.DataFrame(classification_results)


Device set to use mps:0


KeyboardInterrupt: 

NameError: name 'results_df' is not defined

In [10]:

# print(torch.__version__)  # should show 2.6.0 or higher
# import sys
# !{sys.executable} -m pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
# !{sys.executable} -m pip uninstall -y torch torchvision torchaudio
# !{sys.executable} -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu







   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 26.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pandas]2m2/3 [pandas]
